In [3]:
from presidio_analyzer import AnalyzerEngine
analyzer = AnalyzerEngine()
print(analyzer.get_supported_entities(language="en"))

['CRYPTO', 'US_PASSPORT', 'EMAIL_ADDRESS', 'US_DRIVER_LICENSE', 'CREDIT_CARD', 'PHONE_NUMBER', 'URL', 'UK_NHS', 'IBAN_CODE', 'US_SSN', 'LOCATION', 'MEDICAL_LICENSE', 'US_BANK_NUMBER', 'PERSON', 'NRP', 'IP_ADDRESS', 'US_ITIN', 'DATE_TIME']


In [13]:
from PIL import Image
from presidio_image_redactor import ImageRedactorEngine

# Get the image to redact using PIL lib (pillow)
image = Image.open("images/driver_license.png")

# Initialize the engine
engine = ImageRedactorEngine()

# Redact the image with pink color
redacted_image = engine.redact(image, language ="en")

# save the redacted image 
redacted_image.save("images/redacted_new.png")
# uncomment to open the image for viewing
# redacted_image.show()

In [10]:
import pytesseract
pytesseract.pytesseract.tesseract_cmd = "/home/long_2/hai/fastapi_docker/.venv/bin/tesseract.AppImage"
print(pytesseract.get_tesseract_version())

5.5.1
